In [ ]:
import tifffile as tiff
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
from scipy import ndimage
import SimpleITK as sitk
from skimage.filters import threshold_otsu
from skimage import measure

In [ ]:
folder_xct = Path(r'E:\temporal\volumenes\2+3+4+5\pegatinas')

filename_xct = folder_xct / 'frontal_90right.tif'

output_folder = folder_xct / 'output'

output_folder.mkdir(exist_ok=True)

xct = tiff.imread(filename_xct)

print(xct.shape)

folder_ut = Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\Pruebas Pegatinas\doble cara 2')

filename_ut = folder_ut / 'aligned.tif'

output_folder.mkdir(exist_ok=True)

ut = tiff.imread(filename_ut)
ut = np.swapaxes(ut, 0, 1)
ut = np.swapaxes(ut, 1, 2)

print(ut.shape)

In [ ]:

def resize_image(original_image, size, show = False):
    width, height = original_image.size
    if show:
        print(f"The original image size is {width} wide x {height} tall")

    resized_image = original_image.resize(size)
    width, height = resized_image.size
    if show:
        print(f"The resized image size is {width} wide x {height} tall")
    return np.array(resized_image)

def calculate_new_dimensions(original_resolution, new_resolution, original_dimensions):
    # Calculate the original dimensions in real-world units
    original_width, original_height = original_dimensions
    real_world_width = original_width * original_resolution
    real_world_height = original_height * original_resolution

    # Calculate the new dimensions in pixels
    new_width = int(real_world_width / new_resolution)
    new_height = int(real_world_height / new_resolution)

    return new_width, new_height

def get_brightest(d):
    data = d.copy()
    #turn to 0 the values below 255
    data[data < 255] = 0
    #calculate the max of each image
    max_values = np.sum(data, axis=(1, 2))
    
    return np.argmax(max_values)



In [ ]:
def plot_images(images, figsz = (5, 5)):
    fig, axs = plt.subplots(1, len(images), figsize=figsz)
    for i, img in enumerate(images):
        axs[i].imshow(img, cmap='gray')
        axs[i].axis('off')
    plt.show()

def circles(img):
    # Convert binary image to grayscale
    img = (img * 255).astype(np.uint8)

    # Find contours
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an empty image to draw circles on
    circle_img = np.zeros_like(img)

    # For each contour
    for contour in contours:
        # Find minimum enclosing circle
        (x, y), radius = cv2.minEnclosingCircle(contour)
        # Draw the circle
        cv2.circle(circle_img, (int(x), int(y)), int(radius), (255, 255, 255), -1)

        break

    return circle_img

def rectangles(img, thickness=2):
    # Convert binary image to grayscale
    img = (img * 255).astype(np.uint8)

    # Find contours
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an empty image to draw rectangles on
    rectangle_img = np.zeros_like(img)

    # For each contour
    for contour in contours:
        # Find bounding rectangle
        x, y, w, h = cv2.boundingRect(contour)
        # Draw the rectangle contour
        cv2.rectangle(rectangle_img, (x, y), (x+w, y+h), (255, 255, 255), thickness)

    return rectangle_img > 0

import cv2
import numpy as np

def draw_bounding_box(image):

    # Convert binary image to grayscale
    image = (image * 255).astype(np.uint8)

    # Create a new binary image of the same size
    output_image = np.zeros_like(image)

    # Find contours in the image
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # For each contour, draw the bounding box on the new image
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(output_image, (x, y), (x+w, y+h), (255), -1)

    return output_image > 0

def find_rectangle_centers(image):
    # Ensure the image is binary
    assert np.array_equal(image, image.astype(bool)), "Image must be binary"

    # Find connected components in the image
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(image.astype('uint8'))

    # The first component is the background, so ignore it
    return centroids[1:]

def paint_points_on_image(points, image):
    # Create a copy of the image to avoid modifying the original
    image_copy = np.copy(image)

    # Convert the image to RGB if it's grayscale
    if len(image_copy.shape) == 2:
        image_copy = np.stack((image_copy,)*3, axis=-1)

    # Paint each point in red
    for point in points:
        x, y = point
        image_copy[int(y), int(x)] = [255, 0, 0]  # RGB for red

    return image_copy

In [ ]:
aligned_point = 75
original_resolution = 0.0218
new_resolution = 1

gate1 = (aligned_point-1, aligned_point+1)

In [ ]:

id = get_brightest(xct)

original_dimensions = (xct.shape[2], xct.shape[1])
new_dimensions = calculate_new_dimensions(original_resolution, new_resolution, original_dimensions)
print(f"The new dimensions are {new_dimensions}")
resized = resize_image(Image.fromarray(xct[id]), new_dimensions)

#resized the whole volume
resized_xct = np.zeros((xct.shape[0], new_dimensions[1], new_dimensions[0]), dtype=np.uint8)
for i, img in enumerate(xct):
    resized_xct[i] = resize_image(Image.fromarray(img), new_dimensions)

print(resized_xct.shape)

thresh = threshold_otsu(resized)

thresholded = resized > thresh

#delete smallest region

# Label all distinct regions in the binary image
labels = measure.label(thresholded)

# Get properties of each region
regions = measure.regionprops(labels)

# Find the smallest region by area
min_region = min(regions, key=lambda region: region.area)

# Set all pixels in the smallest region to 0
thresholded[labels == min_region.label] = 0

thresholded[labels == 1] = 0
thresholded[labels == 2] = 0
thresholded[labels == 3] = 0

xct_boxes = (draw_bounding_box(thresholded)*255)

xct_centers = centers = find_rectangle_centers(xct_boxes>0)

plot_images([resized, thresholded, xct_boxes, paint_points_on_image(xct_centers, resized)])

In [ ]:

#get the gates
gated_ut = ut[:,:,gate1[0]:gate1[1]]

#get the max projection of gated 1 data
max_projection = np.max(gated_ut, axis=2)

#apply canny edge detection
edges = cv2.Canny(max_projection, 50, 150)

edges = edges > 0

#fill holes in edges
edges_closed = cv2.morphologyEx(edges.astype(np.uint8), cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5)))

rectangle = rectangles(edges,7)

#nand the edges and the rectangle
only_circles = np.logical_and(edges, np.logical_not(rectangle))

#fill the holes in only circles using ndimage
only_circles_filled = ndimage.binary_fill_holes(only_circles)

boxes = (draw_bounding_box(only_circles_filled)*255)

#delete smallest region

# Label all distinct regions in the binary image
labels = measure.label(boxes)

# Get properties of each region
regions = measure.regionprops(labels)

boxes[labels == 1] = 0
boxes[labels == 2] = 0
boxes[labels == 3] = 0

centers = find_rectangle_centers(boxes>0)

print(centers)

plot_images([edges,edges_closed,rectangle,only_circles,only_circles_filled,boxes,resized,xct_boxes])

In [ ]:
fixed_image = sitk.GetImageFromArray(boxes)
moving_image = sitk.GetImageFromArray(xct_boxes)

# Initial alignment of the two volumes
initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                    moving_image, 
                                                    sitk.Euler2DTransform(), 
                                                    sitk.CenteredTransformInitializerFilter.GEOMETRY)
# Set up the registration framework
registration_method = sitk.ImageRegistrationMethod()

# Similarity metric settings
registration_method.SetMetricAsMattesMutualInformation()

# Interpolator
registration_method.SetInterpolator(sitk.sitkLinear)

# Optimizer settings
registration_method.SetOptimizerAsRegularStepGradientDescent(learningRate=1.0, minStep=1e-5, numberOfIterations=500, relaxationFactor=0.5, gradientMagnitudeTolerance=1e-8, estimateLearningRate=registration_method.EachIteration)
registration_method.SetOptimizerScalesFromPhysicalShift()

# Setup for the multi-resolution framework
registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4, 2, 1])
registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas = [2, 1, 0])
registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

# Don't optimize in-place, we would possibly like to run this cell multiple times
registration_method.SetInitialTransform(initial_transform, inPlace=False)


In [ ]:

final_transform = registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32), 
                                                            sitk.Cast(moving_image, sitk.sitkFloat32))

In [95]:
# Now, apply the transform to the top volume overlaping reggion
resampler = sitk.ResampleImageFilter()
resampler.SetTransform(final_transform)
resampler.SetDefaultPixelValue(0)  

# Set the properties of the resampler to match the original top volume
resampler.SetOutputSpacing(fixed_image.GetSpacing())
resampler.SetSize(fixed_image.GetSize())
resampler.SetOutputDirection(fixed_image.GetDirection())
resampler.SetOutputOrigin(fixed_image.GetOrigin())
resampler.SetDefaultPixelValue(fixed_image.GetPixelIDValue())

# Apply the transformation
registered = resampler.Execute(moving_image)

resized_registered = resampler.Execute(sitk.GetImageFromArray(resized))

#resized the whole volume
registered_volume = np.zeros((resized_xct.shape[0],boxes.shape[0], boxes.shape[1]), dtype=np.uint8)
print(registered_volume.shape)
for i, img in enumerate(resized_xct):
    a = resampler.Execute(sitk.GetImageFromArray(resized_xct[i]))
    a = sitk.GetArrayFromImage(a)
    registered_volume[i] = a

(300, 164, 78)


In [ ]:
plot_images([sitk.GetArrayFromImage(fixed_image),sitk.GetArrayFromImage(registered),sitk.GetArrayFromImage(resized_registered)])

In [96]:
#save the fixed image
tiff.imsave(output_folder / 'fixed.tif', sitk.GetArrayFromImage(fixed_image))
#save the registered image
tiff.imsave(output_folder / 'registered.tif', sitk.GetArrayFromImage(registered))
#save the resized registered image
tiff.imsave(output_folder / 'resized_registered.tif', sitk.GetArrayFromImage(resized_registered))
#save the registered volume
tiff.imsave(output_folder / 'registered_volume.tif', registered_volume)

C:\Users\alberto.vicente\AppData\Local\Temp\ipykernel_24108\436565323.py:2: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(output_folder / 'fixed.tif', sitk.GetArrayFromImage(fixed_image))
C:\Users\alberto.vicente\AppData\Local\Temp\ipykernel_24108\436565323.py:4: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(output_folder / 'registered.tif', sitk.GetArrayFromImage(registered))
C:\Users\alberto.vicente\AppData\Local\Temp\ipykernel_24108\436565323.py:6: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(output_folder / 'resized_registered.tif', sitk.GetArrayFromImage(resized_registered))
C:\Users\alberto.vicente\AppData\Local\Temp\ipykernel_24108\436565323.py:8: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(output_folder / 'registered_volume.tif', registered_volume)


Mañana tienes que leer lo del correo de Fede.
Despues probar la robustez de los cuadrados.
Despues hacer lo de el punto medio en vez del local minima porque pinta que eso no va a funcionar